In [1]:
import arcpy

# set initial workspace
arcpy.env.workspace = r"C:\Users\cheri\Documents\geog778\ResidentialUnitTimeAnalysis\ResidentialUnitTimeAnalysis.gdb"

In [13]:
# create lists of the annual variable fields

# attribute_name = ['resunits', 'nonResSF', 'PYR_MARKET', 'PYR_TAXES', 'homesteads']
attribute_names = ['nonResSF', 'PYR_MARKET',   'PYR_TAXES', 'homestead'] # already did 'resunits' manually 

In [12]:
# For each variable of interest, transpose the table to pull out one row for each year for each hexagram. This will produce a separate table for each variable.

for name in attribute_names:
    in_table_transp = r"C:\Users\cheri\Documents\geog778\ResidentialUnitTimeAnalysis\hexBin_2022_3.gdb\hexBin_2022GDB_noSL"
    in_field_transp = f'{name}_2009_NU 2009; {name}_2010_NU 2010; {name}_2011_NU 2011; {name}_2012_NU 2012; {name}_2013_NU 2013; {name}_2014_NU 2014; {name}_2015_NU 2015; {name}_2016_NU 2016; {name}_2017_NU 2017; {name}_2018_NU 2018; {name}_2019_NU 2019; {name}_2020_NU 2020; {name}_2021_NU 2021; {name}_2022_NU 2022'
    out_table_transp = f'hexBin_2022GDB_Transp_{name}' 
    in_transposed_field_name = 'year' 
    in_value_field_name = f'{name}'
    attribute_fields = ['SHAPE', 'OBJECTID']
    
    arcpy.management.TransposeFields(in_table_transp, in_field_transp, out_table_transp, in_transposed_field_name, in_value_field_name, attribute_fields)

In [15]:
# WORKS
# For each new table, calculate a new field by copying 'year' into a new date type field. 

for name in attribute_names:
    in_table_date = f'hexBin_2022GDB_Transp_{name}'
    input_time_field = 'year'
    input_time_format = "yyyy"
    output_time_field = "yearDate"
    output_time_type = "DATE"

    arcpy.management.ConvertTimeField(in_table_date, input_time_field, input_time_format, output_time_field, output_time_type)

In [16]:
# WORKS
# Space Time Cube requires a number variable to go into each bin, so we first need to convert the attribute field into a number type

for name in attribute_names:
    inFeatures_num = f'hexBin_2022GDB_Transp_{name}'
    field_name_num = f'{name}NUM'
    expression_num = f'!{name}!'

    arcpy.management.CalculateField(inFeatures_num, field_name_num, expression_num, "PYTHON3", field_type='FLOAT')

In [21]:
# create space time cube

for name in attribute_names:

    in_features_cube = f'hexBin_2022GDB_Transp_{name}'
    output_cube = rf'C:\Users\cheri\Documents\geog778\ResidentialUnitTimeAnalysis\{name}_STCube.nc'
    location_id = 'OBJECTID_1'
    temporal_aggregation = 'NO_TEMPORAL_AGGREGATION'
    time_field = 'yearDate'
    varsOfInterest = f'{name}NUM SPACE_TIME_NEIGHBORS'    

    arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(in_features_cube, output_cube, location_id, temporal_aggregation, time_field, time_step_interval= '1 Years', time_step_alignment='END_TIME', variables=varsOfInterest)

    # arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(in_features, output_cube, location_id, temporal_aggregation, time_field, {time_step_interval}, {time_step_alignment}, {reference_time}, {variables}, {summary_fields}, {in_related_table}, {related_location_id})

In [24]:
# For each Space Time Cube, do an emerging hotspot/cold spot analysis.

for name in attribute_names:

    in_cube = rf'C:\Users\cheri\Documents\geog778\ResidentialUnitTimeAnalysis\{name}_STCube.nc'
    analysis_variable = f'{name.upper}NUM_NONE_SPACE_TIME_NEIGHBORS' #getting error on this because it needs to be all caps. Ran all manually but need to be fixed.
    output_features = f'{name}_STCube_EmergingHotSpotAnalysis'

    arcpy.stpm.EmergingHotSpotAnalysis(in_cube, analysis_variable, output_features, conceptualization_of_spatial_relationships='FIXED_DISTANCE', define_global_window='ENTIRE_CUBE')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 110024: <built-in method upper of str object at 0x000002195734E770>NUM_NONE_SPACE_TIME_NEIGHBORS is not a variable in the Space Time Cube.
Failed to execute (EmergingHotSpotAnalysis).
